# MBTI별 포스팅 모으기 

In [1]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request

In [2]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# tokenize and stemming

In [3]:
X_data = mbti['posts'].ravel()
y_data = mbti['type']


print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    rm_urls = re.sub(r'http[s]?://\S+', '', sentence)
    clean_sentence = re.sub('[^A-Za-z\s]+', '', rm_urls.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [4]:
import nltk
import gc
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stop_words = set(stopwords.words('english'))
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        if word not in stop_words:
            tempData.append(ps.stem(word))
    stemData.append(tempData)
print("stemming 완료")
print(stemData[0])
stemData = np.array(stemData)
#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]
del [[mbti]]
gc.collect()

stemming 시작
stemming 완료
['oi', 'went', 'break', 'month', 'ago', 'togeth', 'year', 'plan', 'life', 'around', 'relationship', 'wasnt', 'one', 'break', 'relationship', 'might', 'imagin', 'ourenfj', 'pun', 'mani', 'punswel', 'person', 'dont', 'go', 'much', 'attract', 'gener', 'see', 'chang', 'that', 'good', 'alreadi', 'may', 'ask', 'want', 'mere', 'sexualsorri', 'infp', 'im', 'realli', 'postrock', 'post', 'go', 'leav', 'sneak', 'thread', 'dthat', 'doesnt', 'sound', 'enfj', 'think', 'x', 'id', 'never', 'act', 'cold', 'toward', 'romant', 'interest', 'get', 'warm', 'side', 'think', 'like', 'someon', 'way', 'want', 'straight', 'themnot', 'realli', 'im', 'mostli', 'guitar', 'player', 'bass', 'obsess', 'im', 'person', 'ask', 'bass', 'louder', 'least', 'relev', 'like', 'tri', 'new', 'instrument', 'sometim', 'add', 'aguitar', 'mostli', 'add', 'bass', 'glockenspiel', 'harmonica', 'drum', 'ocarinaahahah', 'im', 'sorri', 'im', 'laugh', 'hard', 'realli', 'plagu', 'never', 'thought', 'possiblemostli', 

0

# Vectorization (tfidf)

In [6]:
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())
X = tfidf.transform(stemData)
y = np.array(y_data)


[[0.        0.        0.        ... 0.        0.        0.       ]
 [0.0518922 0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 ...
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]


In [7]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

Counter(y_data)
X_resampled, y_resampled = SMOTE(random_state=0).fit_sample(X, y)
#format(Counter(y_resampled))
X_resampled #ndarray
Counter(y_resampled)

Counter({'ENFJ': 1832,
         'ENFP': 1832,
         'ENTJ': 1832,
         'ENTP': 1832,
         'ESFJ': 1832,
         'ESFP': 1832,
         'ESTJ': 1832,
         'ESTP': 1832,
         'INFJ': 1832,
         'INFP': 1832,
         'INTJ': 1832,
         'INTP': 1832,
         'ISFJ': 1832,
         'ISFP': 1832,
         'ISTJ': 1832,
         'ISTP': 1832})

# XGBoost_Oversampling

In [23]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

param = {}
param['learning_rate'] = 0.03
param['n_estimators'] = 500
param['max_depth'] = 10


print("Model building Start")
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size= 0.3, random_state=1234)
model = XGBClassifier(**param)
model.fit(X_train, y_train)

Model building Start


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.03, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [24]:
preds = model.predict(X_test)
print("Model building End")

Model building End


In [25]:
import joblib

joblib.dump(model, 'XGBoost_smote_paramTuned.pkl')

['XGBoost_smote_paramTuned.pkl']

In [26]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Evaluation Start")
# labels과 guesses
labels = preds
guesses = y_test

print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses, average=None))
print(precision_score(labels, guesses, average=None))
print(f1_score(labels, guesses, average=None))

print("Evaluation END")

Evaluation Start
0.9180122811007505
[0.96666667 0.9249531  0.96167883 0.87477954 0.99417476 0.99243856
 1.         0.99061914 0.77163904 0.73735726 0.85370741 0.79893238
 0.98743268 0.93960924 0.97093023 0.94627383]
[0.97178131 0.88035714 0.95818182 0.87787611 0.99417476 0.98499062
 0.9912892  0.98691589 0.77163904 0.79020979 0.81609195 0.79751332
 0.97690941 0.9600726  0.96905222 0.96808511]
[0.96921724 0.9021043  0.95992714 0.87632509 0.99417476 0.98870056
 0.99562555 0.98876404 0.77163904 0.7628692  0.834476   0.79822222
 0.98214286 0.9497307  0.96999032 0.95705521]
Evaluation END


In [27]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_val_score

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
score = cross_val_score(model, X_train, y_train, cv=skf )
print(score)
print(score.mean())

[0.89839181 0.9039961  0.9081384  0.90860346 0.90251036]
0.9043280269892378


# HarryPotter MBTI

In [28]:
import nltk
import numpy as np
import pandas as pd

from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import re
import glob   

'''HP data Normalization and Stemming'''
# bag_of_words = {} #{등장인물 : 해당인물의 stemData, ~}
h_normalized_text = []
h_stemData = []

ps=PorterStemmer()
stop_words = set(stopwords.words('english'))
input_file = r'.\data\\test\\'

allHPfile_list = glob.glob(os.path.join(input_file, 'HP_*'))
print(allHPfile_list)

harryPotter={}
for file in allHPfile_list:
    df = pd.read_csv(file, sep=',', encoding='iso-8859-1')
    harryPotter[df['Character'][0]] = df['Sentence'].ravel()

print("stemming 시작")
for element in harryPotter.keys():
    h_normalized_text = []
    h_stemData = []
    for sentence in harryPotter[element]:
        h_clean_sentence = re.sub('[^A-Za-z\s]+', '', sentence.lower())
        h_normalized_text.append(h_clean_sentence)
    for sentence in h_normalized_text:
        tokenData = nltk.word_tokenize(sentence)
        for word in tokenData:
            if word not in stop_words:
                h_stemData.append(ps.stem(word))
    harryPotter[element] = h_stemData
    print(harryPotter[element])

print("stemming 완료")

['.\\data\\\\test\\HP_DUMBLEDORE.csv', '.\\data\\\\test\\HP_HAGRID.csv', '.\\data\\\\test\\HP_HARRY.csv', '.\\data\\\\test\\HP_HERMIONE.csv', '.\\data\\\\test\\HP_RON.csv']
stemming 시작
['shouldv', 'known', 'would', 'professor', 'mcgonagal', 'im', 'afraid', 'professor', 'good', 'bad', 'hagrid', 'bring', 'ah', 'professor', 'would', 'trust', 'hagrid', 'life', 'problem', 'trust', 'hagrid', 'albu', 'realli', 'think', 'safe', 'leav', 'peopl', 'famili', 'exactli', 'he', 'far', 'better', 'grow', 'away', 'he', 'readi', 'hagrid', 'realli', 'goodby', 'good', 'luckharri', 'potter', 'startofterm', 'notic', 'wish', 'announc', 'first', 'year', 'pleas', 'note', 'dark', 'forest', 'strictli', 'forbidden', 'student', 'also', 'caretak', 'mr', 'filch', 'ask', 'remind', 'thirdfloor', 'corridor', 'right', 'hand', 'side', 'bound', 'everyon', 'wish', 'die', 'pain', 'death', 'thank', 'let', 'feast', 'begin', 'sillllleeennnnceeee', 'everyon', 'pleas', 'panic', 'prefect', 'lead', 'hous', 'back', 'dormitori', 'tea

In [29]:
'''HP data Vectorization'''
# import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer

hx_stemData = []
hy=[]

for element in harryPotter.keys():
    hx_stemData.append(harryPotter[element])
    hy.append(element)
    
HX_data = tfidf.transform(hx_stemData)
print(HX_data)
HY_data = np.array(hy)
print(HY_data)

'''for element in harryPotter.keys():
    tfidf.fit(harryPotter[element])
    script = tfidf.transform(harryPotter[element])
    X.append(script)
    y.append(element)

print(X.shape)'''

  (0, 270465)	0.012374593925889747
  (0, 270313)	0.022067483571677683
  (0, 269219)	0.014240516611604187
  (0, 267474)	0.02772752743732857
  (0, 267390)	0.027800148045295868
  (0, 267218)	0.04303890043535387
  (0, 265241)	0.016152884823988474
  (0, 264536)	0.04853164344289074
  (0, 263989)	0.008332083298832612
  (0, 263234)	0.026885880686819727
  (0, 262838)	0.009405877933025634
  (0, 262522)	0.02772752743732857
  (0, 261643)	0.04260782966050159
  (0, 261600)	0.03895839579006434
  (0, 260910)	0.017162189867542994
  (0, 260615)	0.01663794771972933
  (0, 260005)	0.02155822849472286
  (0, 259793)	0.010564663741238265
  (0, 259670)	0.016716492689515066
  (0, 259478)	0.02411827242598302
  (0, 257091)	0.0561103208790921
  (0, 256899)	0.03990758026737118
  (0, 256138)	0.10335540022094618
  (0, 255442)	0.02316087655645522
  (0, 254487)	0.020540527981028463
  :	:
  (4, 18979)	0.008467296172184155
  (4, 17690)	0.014928755306457308
  (4, 16070)	0.006551198389139308
  (4, 15337)	0.0116507032533767

'for element in harryPotter.keys():\n    tfidf.fit(harryPotter[element])\n    script = tfidf.transform(harryPotter[element])\n    X.append(script)\n    y.append(element)\n\nprint(X.shape)'

In [30]:
import joblib  
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

model = joblib.load('XGBoost_smote_only.pkl')
# xgb_model_loaded.get_xgb_params()    # Get xgboost specific parameters.
harrypotter_predict = model.predict(HX_data)
print(harrypotter_predict)

['INTJ' 'INFJ' 'INFJ' 'INFJ' 'INFJ']
